# This is Review Scraper

In [53]:
movie1 = pd.read_csv('first_328_movies.csv')

In [54]:
movie2 = pd.read_csv('after_328_all_movies_df.csv')

In [55]:
all_movies = pd.concat([movie1, movie2], axis=0)

In [56]:
all_movies.shape

(1014, 15)

In [57]:
all_movies.columns

Index(['Unnamed: 0', 'Movie ID', 'Title', 'IMDB Rating', 'Directors',
       'Writers', 'Stars', 'Storyline', 'Release Date', 'Origin Countries',
       'Languages', 'Budget', 'Gross Worldwide', 'Runtime', 'Genres'],
      dtype='object')

# Running below cell fetch reviews for one particular movie

In [37]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import chromedriver_autoinstaller
import time

# Automatically install the correct version of chromedriver that matches the installed Chrome version
chromedriver_autoinstaller.install()

# Chrome options to set user-agent
chrome_options = ChromeOptions()
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

# Chrome service
chrome_service = ChromeService()

# Initialize Chrome WebDriver
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

# List of URLs (example: movie_links should be defined)
urls = ["https://www.imdb.com/title/tt0266697/reviews?ref_=tt_urv"]

# Iterate over each URL
for url in urls:
    driver.get(url)  # Use driver.get(url) instead of driver.get([url])

    reviews = []

    # Function to extract reviews from the page
    def extract_reviews():
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        review_elements = soup.find_all('div', class_='lister-item-content')

        for review_element in review_elements:
            title = review_element.find('a', class_='title').text.strip()
            date = review_element.find('span', class_='review-date').text.strip()
            description = review_element.find('div', class_='text').text.strip()
            rating_element = review_element.find('span', class_='rating-other-user-rating')
            rating = rating_element.find('span').text.strip() if rating_element else 'N/A'

            reviews.append({
                'movie_id': url.split('/')[4],
                'title': title,
                'date': date,
                'description': description,
                'rating': rating
            })

    # Load more reviews up to 8 times
    max_loads = 7
    for _ in range(max_loads):
        try:
            extract_reviews()
            load_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, 'load-more-trigger'))  # Adjust this selector based on your website's load more button
            )
            load_more_button.click()
            time.sleep(2)  # Wait for the new reviews to load
        except Exception as e:
            print(f"An exception occurred: {e}")
            break

    # Extract reviews from the last loaded page
    extract_reviews()

# Close the WebDriver
driver.quit()

# Convert the reviews to a pandas DataFrame
review_df = pd.DataFrame(reviews)


In [38]:
review_df

,movie_id,title,date,description,rating
0,tt0266697,Tarantino's Blue Period is OVER!,10 October 2003,"Just saw Kill Bill: Vol. 1 and was surprised, ...",9
1,tt0266697,Kill Bill Is One Hell Of A Thrill,10 September 2020,Pros:\n1. The bright colour palette does a won...,9
2,tt0266697,"I love this movie, I don't know why, but I jus...",26 April 2019,"This is movie is unusual, but it works for me....",9
3,tt0266697,Fight,22 June 2020,I like how she wears a yellow tracksuit and ch...,10
4,tt0266697,Cinema has rarely been this exciting!,28 October 2015,"Having seen Tarantino's 3 previous films, goin...",10
...,...,...,...,...,...
895,tt0266697,Designer violence festival from the bucket-of-...,1 October 2005,A woman - with a past in a cartoon style assig...,6
896,tt0266697,Get ready to bathe into blood,10 August 2018,I am speechless how to praise such a real gem ...,10
897,tt0266697,Very bloody...,24 June 2019,"Boy, was there a ton of blood shed in this mov...",7
898,tt0266697,it is kill something allright. but it isn't bill,19 December 2003,******Spoilers*****and a lot of them too****Si...,1


# ⚠️Running below cell will fetch reviews of 1100 movies, which may take around ⚠️6 HOURS to complete the execution⚠️

### instead i have saved this reviews in csv named "imdb_movies_dataset.csv"

In [58]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import chromedriver_autoinstaller
import time

# Automatically install the correct version of chromedriver that matches the installed Chrome version
chromedriver_autoinstaller.install()

# Chrome options to set user-agent
chrome_options = ChromeOptions()
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

# Chrome service
chrome_service = ChromeService()

# Initialize Chrome WebDriver
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)


reviews = []

# Iterate over each movie ID
for movie_id in all_movies['Movie ID']:
    url = f"https://www.imdb.com/title/{movie_id}/reviews?ref_=tt_urv"
    driver.get(url)  # Use driver.get(url) instead of driver.get([url])

    try:
        # Wait until the page title contains "IMDb"
        WebDriverWait(driver, 20).until(EC.title_contains("IMDb"))
    except TimeoutException:
        print(f"Timeout waiting for {url} to load IMDb page title")
        continue  # Skip to the next movie URL or handle the error appropriately

    # Function to extract reviews from the page
    def extract_reviews():
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        review_elements = soup.find_all('div', class_='lister-item-content')

        for review_element in review_elements:
            title = review_element.find('a', class_='title').text.strip()
            date = review_element.find('span', class_='review-date').text.strip()
            description = review_element.find('div', class_='text').text.strip()
            rating_element = review_element.find('span', class_='rating-other-user-rating')
            rating = rating_element.find('span').text.strip() if rating_element else 'N/A'

            reviews.append({
                'movie_id': movie_id,
                'title': title,
                'date': date,
                'description': description,
                'rating': rating
            })

    # Extract the total number of reviews
    try:
        total_reviews_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="main"]/section/div[2]/div[1]/div/span'))
        )
        total_reviews_text = total_reviews_element.text
        total_reviews = int(total_reviews_text.split()[0].replace(',', ''))  # Extract number and remove commas
    except Exception as e:
        print(f"Failed to get the total number of reviews for {url}: {e}")
        total_reviews = 0  # Default to 0 if unable to get the number of reviews

    # Set max_loads based on the number of reviews
    if total_reviews < 1500:
        max_loads = 9
    elif 2000 <= total_reviews <= 2500:
        max_loads = 12
    elif total_reviews > 25000:
        max_loads = 13
    else:
        max_loads = 10  # Default value

    # Load more reviews up to max_loads times
    for _ in range(max_loads):
        try:
            extract_reviews()
            load_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, 'load-more-trigger'))  # Adjust this selector based on your website's load more button
            )
            load_more_button.click()
            time.sleep(2)  # Wait for the new reviews to load
        except Exception as e:
            print(f"An exception occurred while loading more reviews for {url}: {e}")
            break

    # Extract reviews from the last loaded page
    extract_reviews()

# Close the WebDriver
driver.quit()

# Convert the reviews to a pandas DataFrame
review_df = pd.DataFrame(reviews)



An exception occurred while loading more reviews for https://www.imdb.com/title/tt6263850/reviews?ref_=tt_urv: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF605BF3E52+31618]
	(No symbol) [0x00007FF605B6B0B9]
	(No symbol) [0x00007FF605A2888A]
	(No symbol) [0x00007FF605A78524]
	(No symbol) [0x00007FF605A7862C]
	(No symbol) [0x00007FF605ABF787]
	(No symbol) [0x00007FF605A9D14F]
	(No symbol) [0x00007FF605ABCA80]
	(No symbol) [0x00007FF605A9CEB3]
	(No symbol) [0x00007FF605A6A46B]
	(No symbol) [0x00007FF605A6B001]
	GetHandleVerifier [0x00007FF605EFA02D+3202397]
	GetHandleVerifier [0x00007FF605F46A4D+3516285]
	GetHandleVerifier [0x00007FF605F3C4C0+3473904]
	GetHandleVerifier [0x00007FF605CA5D56+760454]
	(No symbol) [0x00007FF605B76B5F]
	(No symbol) [0x00007FF605B71CF4]
	(No symbol) [0x00007FF605B71E82]
	(No symbol) [0x00007FF605B6122F]
	BaseThreadInitThunk [0x00007FFE97AA257D+29]
	RtlUserThreadStart [0x00007FFE99D6AF28+40]

An exception occurred while loading more reviews for https://www

In [59]:
review_df

,movie_id,title,date,description,rating
0,tt0266697,Tarantino's Blue Period is OVER!,10 October 2003,"Just saw Kill Bill: Vol. 1 and was surprised, ...",9
1,tt0266697,Kill Bill Is One Hell Of A Thrill,10 September 2020,Pros:\n1. The bright colour palette does a won...,9
2,tt0266697,"I love this movie, I don't know why, but I jus...",26 April 2019,"This is movie is unusual, but it works for me....",9
3,tt0266697,Fight,22 June 2020,I like how she wears a yellow tracksuit and ch...,10
4,tt0266697,Cinema has rarely been this exciting!,28 October 2015,"Having seen Tarantino's 3 previous films, goin...",10
...,...,...,...,...,...
1139444,tt0049552,Sorry Aldo,12 March 2020,Yep Aldo is a right nice guy but this movie ai...,5
1139445,tt0049552,Falls off the cliff,23 April 2004,****SPOILERS**** Unconvincing and very talky m...,5
1139446,tt0049552,Man On The Run,3 July 2012,An innocent man named James Vanning (Aldo Ray)...,N/A
1139447,tt0049552,Less Than The Sum Of Its Parts,13 November 2003,NIGHTFALL is a modest film noir put together b...,N/A


In [72]:
all_movies.drop('Unnamed: 0', axis=1, inplace=True)

In [75]:
all_movies.reset_index(drop=True, inplace=True)

In [76]:
all_movies.tail(5)

,Movie ID,Title,IMDB Rating,Directors,Writers,Stars,Storyline,Release Date,Origin Countries,Languages,Budget,Gross Worldwide,Runtime,Genres
1009,tt0034248,Suspicion,7.3,​Alfred Hitchcock,"noneWriters, ​Samson Raphaelson, ​Joan Harriso...","noneStars, ​Cary Grant, ​Joan Fontaine, ​Cedri...",A shy young heiress marries a charming gentlem...,Release date,United States,English,"$1,103,000 (estimated)",$19,1 hour 39 minutes,"Film-Noir, Mystery, Thriller"
1010,tt0044136,Tomorrow Is Another Day,7.1,​Felix E. Feist,"noneWriters, ​Art Cohn, ​Guy Endore, noneWriters","noneStars, ​Ruth Roman, ​Steve Cochran, ​Luren...","An ex-convict and a woman fall in love, after ...",Release date,United States,English,NaN,NaN,1 hour 30 minutes,"Crime, Drama, Film-Noir"
1011,tt0048261,Kiss Me Deadly,7.5,​Robert Aldrich,"​Mickey Spillane, ​A.I. Bezzerides","noneStars, ​Ralph Meeker, ​Albert Dekker, ​Pau...",A doomed female hitchhiker pulls Mike Hammer i...,Release date,United States,"English, Italian, Spanish","$410,000 (estimated)","$952,000",1 hour 46 minutes,"Crime, Film-Noir, Mystery"
1012,tt0051207,The Wrong Man,7.4,​Alfred Hitchcock,"noneWriters, ​Maxwell Anderson, ​Angus MacPhai...","noneStars, ​Henry Fonda, ​Vera Miles, ​Anthony...","In 1953, an innocent man named Christopher Ema...",Release date,United States,"English, Italian, Spanish","$1,200,000 (estimated)",$494,1 hour 45 minutes,"Drama, Film-Noir"
1013,tt0049552,Nightfall,7.1,​Jacques Tourneur,"​Stirling Silliphant, ​David Goodis","noneStars, ​Aldo Ray, ​Anne Bancroft, ​Brian K...","Through a series of bizarre coincidences, an a...",Release date,United States,"English, Spanish, Italian",NaN,NaN,1 hour 18 minutes,"Crime, Drama, Film-Noir"


In [79]:
# Step 1: Aggregate reviews in review_df
aggregate_reviews = review_df.groupby('movie_id').agg({
    'title': lambda x: list(x),           # Collect titles into a list
    'date': lambda x: list(x),            # Collect dates into a list
    'description': lambda x: list(x),     # Collect descriptions into a list
    'rating': lambda x: list(x)           # Collect ratings into a list
}).reset_index()

# Renaming columns to avoid confusion
aggregate_reviews.columns = ['movie_id', 'review_titles', 'review_dates', 'review_descriptions', 'review_ratings']

# Step 2: Ensure movie_df's 'Movie ID' is string type to match review_df
all_movies['Movie ID'] = all_movies['Movie ID'].astype(str)

# Step 3: Merging the dataframes on movie_id
merged_df = all_movies.merge(aggregate_reviews, left_on='Movie ID', right_on='movie_id', how='left')

# Dropping the redundant 'movie_id' column from the merged dataframe
merged_df = merged_df.drop(columns=['movie_id'])


### running below cell can take while, depending on the processor

In [87]:
merged_df.to_csv('imdb_movies_dataset.csv')

# This csv file is of 3.49 gb 